# Group expenses

In [1]:
%pip install 'opvious>=0.16.1'

## Problem formulation

Let's formulate 

In [2]:
import opvious.modeling as om

class GroupExpenses(om.Model):
    """A mixed-integer model for finding the optimal way to settle debts within a group"""
    
    members = om.Dimension()  # Participants
    transactions = om.Dimension()  # Expenses
    is_participating = om.Parameter.indicator(transactions, members)  # 1 if a member is involved in a transaction
    payment = om.Parameter.non_negative(transactions, members)  # Amount paid by each member in a transaction
    min_transfer = om.Parameter.non_negative()  # Minimum transfer threshold
    
    def __init__(self):
        super().__init__()
        # Additional amount to be paid from one member to another to achieve fairness
        self.transfer = om.Variable.non_negative(self.members, self.members, qualifiers=['sender', 'recipient'])
        # Indicator variable representing a transfer from one member to another (1 if transfer > 0, 0 otherwise)
        self.is_transferring = om.fragments.ActivationVariable(self.transfer, upper_bound=self.total_payments())
    
    def total_payments(self):
        """Total amount of money paid, used as upper-bound for transfers"""
        return om.total(self.payment(t, m) for t, m in self.transactions * self.members)
    
    def fair_payment(self, t, m):
        """Fair payment in a transaction for a given member"""
        share = self.is_participating(t, m) / om.total(self.is_participating(t, o) for o in self.members)
        return share * om.total(self.payment(t, m) for m in self.members)
    
    @om.constraint
    def zero_sum_transfers(self):
        """After netting transfers, each member should have paid the sum of their fair payments"""
        for m in self.members:
            received = om.total(self.transfer(s, m) for s in self.members)
            sent = om.total(self.transfer(m, r) for r in self.members)
            owed = om.total(self.payment(t, m) - self.fair_payment(t, m) for t in self.transactions)
            yield received - sent == owed
            
    @om.constraint
    def transfer_meets_min(self): 
        """Each positive transfer is at least equal to the minimum threshold"""
        for s, r in self.members * self.members:
            yield self.transfer(s, r) >= self.is_transferring(s, r) * self.min_transfer()
            
    @om.objective
    def minimize_total_transferred(self):
        """First objective: minimize the total amount of money transferred between members"""
        return om.total(self.transfer(s, r) for s, r in self.members * self.members)
    
    @om.fragments.magnitude_variable(members, projection=0, lower_bound=False)
    def max_transfers_sent(self, m):
        """Number of transfers sent by each member"""
        return om.total(self.is_transferring(m, r) for r in self.members)
    
    @om.objective
    def minimize_max_transfers_sent(self):
        """Second objective: minimize the maximum number of transfers sent by any member"""
        return self.max_transfers_sent()


model = GroupExpenses()
model.definition_counts().T # Summary of the model's components

title,GroupExpenses
category,
CONSTRAINT,4
DIMENSION,2
OBJECTIVE,2
PARAMETER,3
VARIABLE,3


For the mathematically inclined, models can generate their LaTeX specification.

In [3]:
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">GroupExpenses</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^v_\mathrm{transfer[sender,recipient]}&: \tau \in \mathbb{R}_+^{M \times M} \\
  \S^v_\mathrm{isTransferring}&: \tau^\mathrm{is} \in \{0, 1\}^{M \times M} \\
  \S^c_\mathrm{isTransferringActivates}&: \forall m, m' \in M, \sum_{t \in T, m'' \in M} p_{t,m''} \tau^\mathrm{is}_{m,m'} \geq \tau_{m,m'} \\
  \S^d_\mathrm{members}&: M \\
  \S^d_\mathrm{transactions}&: T \\
  \S^p_\mathrm{isParticipating}&: p^\mathrm{is} \in \{0, 1\}^{T \times M} \\
  \S^p_\mathrm{payment}&: p \in \mathbb{R}_+^{T \times M} \\
  \S^p_\mathrm{minTransfer}&: t^\mathrm{min} \in \mathbb{R}_+ \\
  \S^c_\mathrm{zeroSumTransfers}&: \forall m \in M, \sum_{m' \in M} \tau_{m',m} - \sum_{m' \in M} \tau_{m,m'} = \sum_{t \in T} \left(p_{t,m} - \frac{p^\mathrm{is}_{t,m}}{\sum_{m' \in M} p^\mathrm{is}_{t,m'}} \sum_{m' \in M} p_{t,m'}\right) \\
  \S^c_\mathrm{transferMeetsMin}&: \forall m, m' \in M, \tau_{m,m'} \geq \tau^\mathrm{is}_{m,m'} t^\mathrm{min} \\
  \S^o_\mathrm{minimizeTotalTransferred}&: \min \sum_{m, m' \in M} \tau_{m,m'} \\
  \S^v_\mathrm{maxTransfersSent}&: \sigma^\mathrm{maxTransfers} \in \mathbb{R}_+ \\
  \S^c_\mathrm{maxTransfersSentUpperBounds}&: \forall m \in M, \sigma^\mathrm{maxTransfers} \geq \sum_{m' \in M} \tau^\mathrm{is}_{m,m'} \\
  \S^o_\mathrm{minimizeMaxTransfersSent}&: \min \sigma^\mathrm{maxTransfers} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

In [4]:
import opvious

_client = opvious.Client.default()

async def optimal_transfers(payments, min_transfer=10, tolerance=0.1):
    """Computes optimal transfers to settle expenses fairly within a group
    
    Args:
        payments: Dataframe of payments indexed by transaction where each column is a group member.
            Each member with a non-zero payment will be considered a participant in the transaction.
        min_transfer: Minimum transfer amount during settlement.
        tolerance: Relative slack bound on the total amount of money transferred during settlement
            used to minimize the number of outbound transfers for any one member. For example, the
            default value of 0.1 will allow transfering up to 10% more overall.
    """
    se = payments.stack()['payment']
    res = await _client.run_solve(
        specification=model.specification(),
        parameters={
            'minTransfer': min_transfer,
            'payment': se,
            'isParticipating': (se > 0).astype(int),
        },
        strategy=opvious.SolveStrategy(  # Multi-objective strategy
            epsilon_constraints=[
                # Within tolerance of the smallest total transfer amount
                opvious.EpsilonConstraint('minimizeTotalTransferred', relative_tolerance=tolerance),
                # Using the smallest possible number of transfers per member
                opvious.EpsilonConstraint('minimizeMaxTransfersSent'),
            ],
            target='minimizeTotalTransferred',  # Final target: minimize total transfer amount
        ),
        assert_feasible=True,
    )
    return res.outputs.variable('transfer').unstack(level=1).fillna(0).round(2)

## Testing

In [5]:
import numpy as np
import pandas as pd

_names = ["emma", "noah", "ava", "liam", "isabella", "sophia", "mason", "mia", "lucas", "amelia"]

def random_data(count=100, seed=16):
    rng = np.random.default_rng(seed)
    tuples = []
    for i in range(count):
        tid = f't{i+1:02}'
        for name in _names:
            if rng.integers(2):
                continue
            tuples.append({
                'transaction': tid, 
                'name': name, 
                'payment': round(100 * rng.random(), 2),
            })
    df = pd.DataFrame(tuples).set_index(['transaction', 'name'])
    return df.unstack().fillna(0)

df = random_data()
df.head()

payment                                                           \
name         amelia    ava   emma isabella   liam  lucas  mason   mia   noah   
transaction                                                                    
t01           80.24   0.00   0.00     0.00   9.41   0.00  87.46  85.4   0.00   
t02            0.00  95.86  69.56    66.33   0.00  29.93  39.49  27.8  15.50   
t03            0.00  13.87   0.00    41.36  38.47   0.00   0.00   0.0   0.00   
t04            0.00   0.00   0.00    14.89   0.00  96.19  86.77   0.0  72.99   
t05            0.00   0.00   0.00     0.00   5.80   0.00   0.00   0.0   0.00   

                    
name        sophia  
transaction         
t01          62.15  
t02           0.00  
t03           0.00  
t04           0.00  
t05           0.00

In [6]:
odf = await optimal_transfers(df, tolerance=0.25)
odf

INFO:opvious.client.handlers:Validated inputs. [parameters=2001]
INFO:opvious.client.handlers:Solving problem... [columns=201, rows=220]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=24, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=96, gap=13.77%]
INFO:opvious.client.handlers:Solve in progress... [iterations=331, gap=0.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=1190.0525515873019]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=71, gap=inf]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=1]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=20, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=831, gap=12.86%]
INFO:opvious.cl

value                                      
recipient    emma isabella   liam  lucas   mason sophia
sender                                                 
amelia       0.00     0.00   0.00  31.02    0.00    0.0
ava          0.00     0.00   0.00   0.00    0.00  481.2
emma         0.00    19.97   0.00   0.00    0.00    0.0
liam         0.00    14.06   0.00   0.00    0.00    0.0
lucas        0.00    11.85   0.00   0.00    0.00    0.0
mason        0.00   142.47   0.00   0.00    0.00    0.0
mia        125.52     0.00   0.00   0.00    0.00    0.0
noah         0.00     0.00   0.00   0.00  552.31    0.0
sophia       0.00     0.00  63.94   0.00    0.00    0.0

Next steps:

+ Partial group transactions (formulated [here](https://github.com/opvious/examples/blob/main/sources/group-expenses.md) for example)
+ Different objective ordering
+ 

In [7]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [8]:
logging.getLogger().setLevel(level=logging.INFO)